# Module 9: Cross Account Access - Consumer Account Utilization

---

# Contents

1. [Overview](#Overview)
1. [Setup](#Setup)
1. [Accept pending RAM invitations](#Accept-pending-RAM-invitations)
1. [Testing Discoverability](#Testing-Discoverability)
1. [Testing Access permissions](#Testing-Access-permissions)

# Overview

Amazon SageMaker Feature Store now makes it easier to share, discover and access feature groups
across AWS accounts. This new capability promotes collaboration and minimizes duplicate work for
teams involved in ML model and application development, particularly in enterprise environments
with multiple accounts spanning different business units or functions.

With this launch, account owners can grant access to select feature groups by other accounts using
AWS Resource Access Manager (RAM). Once granted access, users of those accounts can
conveniently view all of their feature groups, including the shared ones, through Amazon SageMaker
Studio or SDKs. This enables teams to discover and utilize features developed by other teams,
fostering knowledge sharing and efficiency. 

In this notebook to be run within the consumer account, you will learn:
- how to accept RAM invitations to discover and access cross account features groups from the admin/owner level
- how to discover cross account features groups which are on the admin/owner account and list these on the consumer account
- how to access in ReadWrite cross account features groups which are on the admin/owner account and perform the following operations from the consumer account: describe(), get_record(), ingest() and delete_record().

Note: It is crucial to ensure proper AWS IAM permissions for using RAM within SageMaker, for successful execution.

# Setup

#### IAM Roles

If you are running this notebook in Amazon SageMaker Studio, the IAM role assumed by your Studio user needs permission to perform RAM operations. To provide this permission to the role, do the following:

1. Open the [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/).
2. Select Amazon SageMaker Studio and choose your user name.
3. Under **User summary**, copy just the name part of the execution role ARN 
4. Go to the [IAM console](https://console.aws.amazon.com/iam) and click on **Roles**. 
5. Find the role associated with your SageMaker Studio user
6. Under the Permissions tab, click **Attach policies** and add the following: **AWSResourceAccessManagerFullAccess**
7. Under Trust relationships, click **Edit trust relationship** and add the following JSON,
```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "sagemaker.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "ram.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```
 

#### Imports

Below cells imports necessary libraries, including boto3, which is used for AWS services, and SageMaker components. It also initialize the logging.

In [ ]:
import boto3
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.feature_group import FeatureGroup
import sagemaker
import logging
import pandas as pd

In [ ]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())
logger.info(f'Using SageMaker version: {sagemaker.__version__}')
logger.info(f'Using Pandas version: {pd.__version__}')

#### Essentials

This cell sets up the AWS environment and initializes key components. The script configures the region, creates a session, and defines a role using get_execution_role() to obtain the SageMaker execution role.

In [ ]:
region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name="sagemaker")
ram_client = boto3.client("ram")
featurestore_runtime = boto_session.client(service_name="sagemaker-featurestore-runtime")
feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

role = get_execution_role()

# Accept pending RAM invitations

The script below is listing all the invitations in pending state and accepting all

You can as well check the RAM console if there is any pending requests that's initiated by admin
And then accept the RAM invitation for feature groups manually from the RAM console

In [ ]:
response = ram_client.get_resource_share_invitations()
if 'resourceShareInvitations' in response:
    for inv in response["resourceShareInvitations"]:
        if inv["status"] == 'PENDING':
            acct_response = ram_client.accept_resource_share_invitation(
                resourceShareInvitationArn=inv["resourceShareInvitationArn"]
            )
            logger.info(f'Resource-share-invitation accepted for:: {inv["resourceShareInvitationArn"]}')
            logger.info(acct_response)

# Testing Discoverability

Discoverability means being able to see feature group names and metadata. When you grant
discoverability permission, all feature group entities in the account that you share from
(resource owner account) become discoverable by the accounts that you are sharing with
(resource consumer account). 

In below cell we search for the features groups that were created in the admin/owner account and we display their names and ARNs.

In [ ]:
# As the default catalog is shared, consumer can be able to see all cross account shared resources
search_response = sagemaker_client.search(
    Resource = "FeatureGroup",
    CrossAccountFilterOption = 'CrossAccount'
)

# Owner features groups
feature_group_name_1 = 'cross-account-fg-1'
feature_group_name_2 = 'cross-account-fg-2'

if 'Results' in search_response:
    for fg in search_response["Results"]:
        if fg["FeatureGroup"]["FeatureGroupName"] == feature_group_name_1:
            feature_group_name_arn_1 = fg["FeatureGroup"]["FeatureGroupArn"]
            logger.info(f'FG ARN: {feature_group_name_arn_1}')
        if fg["FeatureGroup"]["FeatureGroupName"] == feature_group_name_2:
            feature_group_name_arn_2 = fg["FeatureGroup"]["FeatureGroupArn"]      
            logger.info(f'FG ARN: {feature_group_name_arn_2}')

# Testing Access permissions

When you grant an access permission, you do so at a feature group resource level (not at
account level). This gives you more granular control over granting access to data. The type of
access permissions that can be granted are: read-only, read-write, and admin. 

In this cell, the script is trying to identify a pre-existing Feature Group by specifying its ARN (Amazon Resource Name). The Feature Group's ARN is a unique identifier that points to a specific AWS resource. In this case, it is trying to instantiate a FeatureGroup object with the specified ARN for further operations.

In [ ]:
feature_group_1 = FeatureGroup(name=feature_group_name_arn_1, sagemaker_session=feature_store_session)
feature_group_2 = FeatureGroup(name=feature_group_name_arn_2, sagemaker_session=feature_store_session)

### Describe a cross account Feature Group

The script attempts to retrieve information about the Feature Group using the describe() method, which provides details about the Feature Group's configuration and status.

This cell is describing the Feature Group 'cross-account-fg-1' and we are successful since we provided Read access to this FG

In [ ]:
fg_desc_1 = feature_group_1.describe()
logger.info(f'FG 1 Description: {fg_desc_1}')

For our seconf FG 'cross-account-fg-2' we get an 'AccessDeniedException' which is normal since we did not provide Read access to this FG

In [ ]:
fg_desc_2 = feature_group_2.describe()
logger.info(f'FG 2 Description: {fg_desc_2}')

### Get a record from a cross account Feature Group

This cell retrieves a specific record from the Feature Group 'cross-account-fg-1' using the get_record() method, which takes a record identifier value as input. In this case, it attempts to retrieve a record with a record_identifier_value of '1'.

In [ ]:
feature_group_1.get_record(record_identifier_value_as_string='1')

For our second FG 'cross-account-fg-2' we get an 'AccessDeniedException' which is normal since we did not provide read access to this FG

In [ ]:
feature_group_2.get_record(record_identifier_value_as_string='1')

### Ingest records into a cross account Feature Group

This cell is responsible for creating a Pandas DataFrame named new_data_df. 

In [ ]:
# Assuming 'new_data' is the data you want to ingest
new_data = [[8, 187512354.0, 160, 240],
            [9, 187512355.0, 140, 210],
            [10, 187512356.0, 170, 280],
            [11, 187512357.0, 155, 260]]

It populates this DataFrame with sample data, including columns like record_id, event_time, feature_11, and feature_12. The data will be used for ingestion into the Feature Group 'cross-account-fg-1'. The operation is successful because we provided write access to FG 'cross-account-fg-1'.

In [ ]:
# Define a DataFrame using the data
new_data_df_1 = pd.DataFrame(new_data, columns=["record_id", "event_time", "feature_11", "feature_12"])

# Ingest data into the cross account feature group 'cross-account-fg-1'
logger.info(f'Preparing to ingest data into feature group: {feature_group_name_1} ...')
feature_group_1.ingest(data_frame=new_data_df_1, max_processes=4, wait=True)
logger.info(f'{len(new_data)} records ingested into feature group: {feature_group_name_1}')

For our second FG 'cross-account-fg-2' we get an 'AccessDeniedException' which is normal since we did not provide write access to this FG.

In [ ]:
# Define a DataFrame using the data
new_data_df_2 = pd.DataFrame(new_data, columns=["record_id", "event_time", "feature_21", "feature_21"])

# Ingest data into the cross account feature group 'cross-account-fg-2'
logger.info(f'Preparing to ingest data into feature group: {feature_group_name_2} ...')
feature_group_2.ingest(data_frame=new_data_df_2, max_processes=4, wait=True)
logger.info(f'{len(new_data)} records ingested into feature group: {feature_group_name_2}')

### Delete record from a cross account Feature Group

The script attempts to delete a specific record from the Feature Group using the delete_record() method. It specifies the record identifier value and event time for the deletion. The operation is successful because we provided write access to FG 'cross-account-fg-1'.

In [ ]:
# Define the values for the record you want to delete
record_identifier_value = '1'
event_time_value = '187512346.0'

# Delete the record from the feature group
feature_group_1.delete_record(
    record_identifier_value_as_string=record_identifier_value,
    event_time=event_time_value
)

For our second FG 'cross-account-fg-2' we get an 'AccessDeniedException' which is normal since we did not provide write access to this FG.

In [ ]:
# Delete the record from the feature group
feature_group_2.delete_record(
    record_identifier_value_as_string=record_identifier_value,
    event_time=event_time_value
)